# 📊 Market Analyzer on Google Colab

このノートブックは、Renderなどの無料枠でメモリ不足になる場合に、Google Colabの強力なリソース（GPU/TPU不要、メモリ12GB〜）を使って分析を実行するためのものです。

### 準備するもの
1. **Googleアカウント**
2. **APIキー** (Gemini, NewsAPI等)
3. **Githubリポジトリ** (このノートブックが含まれているリポジトリ)
   - **プライベートリポジトリの場合**: GitHubのPersonal Access Token (`GITHUB_TOKEN`) が必要です。

## 1. Google Driveのマウント (データベース保存用)
分析結果のデータベース(`market_analyzer.db`)をGoogle Driveに保存して永続化します。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. リポジトリのクローン & セットアップ
最新のコードを取得し、ライブラリをインストールします。

### リポジトリの取得設定
- リポジトリが **Private** の場合、Colabのサイドバー「🔑 (Secrets)」に `GITHUB_TOKEN` を追加してください。
- `GITHUB_TOKEN` が設定されている場合、自動的にそれを使用して認証付きクローンを行います。

In [ ]:
# リポジトリ設定
REPO_USER = "ikaruga1123"
REPO_NAME = "market_analyzer"
REPO_URL_PUBLIC = f"https://github.com/{REPO_USER}/{REPO_NAME}.git"

import os
from google.colab import userdata

# 認証トークンの確認 (Private Repo対応)
repo_url = REPO_URL_PUBLIC
try:
    token = userdata.get('GITHUB_TOKEN')
    if token:
        repo_url = f"https://{token}@github.com/{REPO_USER}/{REPO_NAME}.git"
        print("🔒 Private Repository mode: Using GITHUB_TOKEN for authentication")
    else:
        print("globe Public Repository mode: No GITHUB_TOKEN found")
except Exception:
    print("globe Public Repository mode: No GITHUB_TOKEN found")

# クローンまたはプル
target_dir = f"/content/{REPO_NAME}"
if not os.path.exists(target_dir):
    print(f"📥 Cloning repository to {target_dir}...")
    !git clone {repo_url}
else:
    print(f"🔄 Updating repository in {target_dir}...")
    %cd {target_dir}
    !git pull

%cd {target_dir}

# 依存ライブラリのインストール
print("📦 Installing dependencies...")
!pip install -r requirements.txt
!pip install google-generativeai --upgrade

## 3. 環境変数の設定 (Secrets)
Colabのサイドバーにある「🔑 (Secrets)」から以下のキーを設定してください。
- `GEMINI_API_KEY`
- `NEWSAPI_KEY`
- `NEWSDATA_KEY`
- `MARKETAUX_KEY`
- `SMTP_HOST` (例: smtp.gmail.com)
- `SMTP_PORT` (例: 587)
- `SMTP_USER`
- `SMTP_PASSWORD`
- `SMTP_RECIPIENT`

設定後、以下のセルを実行して読み込みます。

In [ ]:
from google.colab import userdata
import os

# Secretsから読み込み
keys = [
    "GEMINI_API_KEY", "NEWSAPI_KEY", "NEWSDATA_KEY", "MARKETAUX_KEY",
    "SMTP_HOST", "SMTP_PORT", "SMTP_USER", "SMTP_PASSWORD", "SMTP_RECIPIENT"
]

for key in keys:
    try:
        os.environ[key] = userdata.get(key)
        print(f"✅ {key} loaded")
    except Exception:
        print(f"⚠️ {key} not found in Secrets. Skipping.")

# データベースの保存先をGoogle Driveに設定
# Drive上の MyDrive/market_analyzer.db に保存されます
db_path = "/content/drive/MyDrive/market_analyzer.db"
os.environ["DATABASE_URL"] = f"sqlite:///{db_path}"
print(f"📂 Database configured: {db_path}")

## 4. 分析実行 🚀
市場分析を実行し、メールを送信します。

In [ ]:
!python scripts/run_daily.py